In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib
import pandas as pd
import logging
import datetime

In [92]:
def request_url(departure_date):
    url_vairance = {
        'domain':'https://fly.vietnamairlines.com/dx/VNDX/#/flight-selection?',
        'variances': {
            'journeyType': 'one-way',
            # Location to show languace
            'locale': 'en-US',
            # Arrive location
            'origin': 'SGN',
            # Departure location
            'destination': 'NRT',
            # Adult numbers
            'ADT': 1,
            # Not clearly
            'CHD': 0,
            # Not cleary
            'INT': 0,
            # Arrive date
            'date': departure_date,
        }
    }
    return url_vairance['domain'] + urllib.parse.urlencode(url_vairance['variances'])

def general_date(from_date, to_date):
    return [date.strftime('%m-%d-%Y') for date in pd.date_range(from_date, to_date)]

def general_csv_name(now):
    return str(now.year) + str(now.month) + str(now.day) + str('_') + str(now.hour) + str(now.minute) + str('.csv')

## General paramecter

In [94]:
date_range = general_date('12-01-2018', '12-30-2018')

collected_data = {
    'journeyType':[],
    'departure':[],
    'destination':[],
    'search_date':[],
    'departure_datetime':[],
    'arrival_datetime':[],
    'stops':[],
    'passenger':[],
    'prices':[],
    'flight_number':[],
}

def insert(col_name, value):
    collected_data[col_name].append(value)
    
for search_date in date_range:
    journeyType = 'one-way'
    departure = 'NRT'
    destination = 'SGN'
    passenger = 1

    url = request_url(search_date)

    driver = webdriver.Chrome(r'D:\02_Python\18_PullingData_HTML\chormdriver\chromedriver.exe')
    driver.implicitly_wait(20)
    driver.get(url)

    try:
        # Step 1. Get main price table
        flights_main = driver.find_element_by_class_name('flights-table')

        # Step 2. Get children
        flights = flights_main.find_elements_by_class_name('dxp-flight')

        for flight in flights:
            insert('journeyType', journeyType)
            insert('departure', departure)
            insert('destination', destination)
            insert('search_date', search_date)

            depart_arrival_time = flight.find_elements_by_class_name('dxp-time')
            insert('departure_datetime', depart_arrival_time[0].get_attribute('datetime'))
            insert('arrival_datetime', depart_arrival_time[1].get_attribute('datetime'))

            insert('stops', flight.find_element_by_xpath("//td[@class='column flight-stops']").text)
            insert('passenger', passenger)

            prices = []
            for price in flight.find_elements_by_class_name("price-container"):
                prices.append(price.text)    
            insert('prices', prices)

            flight_number = flight.find_element_by_class_name("flight-number").text
            insert('flight_number', flight_number)  
    except ValueError:
        logging.basicConfig(format='%(asctime)s %(message)s')
        logging.warning('Elements is not founded.')
    driver.quit()

In [95]:
collected_pd = pd.DataFrame(collected_data)
now = datetime.datetime.now()
collected_pd['search_time'] = [now] * len(collected_pd['departure'])
collected_pd.to_csv(general_csv_name(now))